# 401 First guess for long-term CDR needs

In this notebook, we create a lookup table to get a first estimate of the CDR needs for drawdown.

In [1]:
import pyam 
import pandas as pd
import numpy as np

import dotenv
import os
from pathlib import Path

%load_ext nb_black
%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Step 1: Read in the lookup table generated in the notebook `304_compile_metrics.ipynb`

In [2]:
metrics = pd.read_csv(
    Path(
        '../data/304_compiled_metrics.csv'
    )
)

<IPython.core.display.Javascript object>

In [3]:
metrics.head(2)

,model,scenario,run_id,drawdown_co2,ZEC,change_non_co2,2015_warming,2100_warming,peak_warming,year_peak_warming,cum_emissions_to_peak_GtCO2,cum_emissions_after_peak_GtCO2,eTCREup,eTCREdown
0,AIM/CGE 2.1,CD-LINKS_NPi2020_400,0,0.153213,-0.086132,0.024094,1.118762,1.402425,1.625633,2036,656.867099,-284.181724,0.000772,-0.000539
1,AIM/CGE 2.1,CD-LINKS_NPi2020_400,1,-0.025027,-0.016988,0.054163,1.411990,2.513044,2.516868,2036,656.867099,-284.181724,0.001682,0.000088


<IPython.core.display.Javascript object>

In [4]:
metrics.set_index(
    [
        'model',
        'scenario',
        'run_id'
    ],
    inplace=True
)

<IPython.core.display.Javascript object>

Step 2: Calculate the desired long-term cooling necessary to hit 1.5 in 2100. For this, we calculate the difference between 1.5 and the 2100 warming.

In [5]:
metrics.loc[:,'cooling_to_1p5'] = (
    1.5
    -
    metrics.loc[:, '2100_warming']
)

<IPython.core.display.Javascript object>

Step 3: Some of these may be positive because peak warming is below 1.5. We want to set these estimates to 0.

In [6]:
metrics.loc[:, 'cooling_to_1p5'] = (
    metrics.loc[:,'cooling_to_1p5'].apply(
        lambda x: x if x < 0 else 0
    )
)

<IPython.core.display.Javascript object>

Step 4: We can now calculate the first guess of the amount of cdr that may be required. However, the eTCREdown does not always show a "cooling" sign. To address this, we use a two tiered approach:
1. Use eTCREdown if it is negative
2. Use - (eTCREup) if eTCREdown is positive
Since this is only for a first guess, we can assume that this error will be corrected for in our iterative procedure later on.

In [7]:
metrics.loc[:, 'eTCREdown_first_guess'] = (
    metrics.apply(
        lambda x: x['eTCREdown'] if x['eTCREdown'] < 0 else -x['eTCREup'],
        axis=1
    )
)

<IPython.core.display.Javascript object>

Step 5: Calculate the additional CDR needs (first guess)

In [8]:
metrics.loc[:, 'additional_cdr_gtco2_first_guess'] = (
    metrics.loc[:, 'cooling_to_1p5']
    /
    metrics.loc[:, 'eTCREdown_first_guess'] 
).round(0)

<IPython.core.display.Javascript object>

In [9]:
metrics

drawdown_co2       ZEC  \
model             scenario             run_id                           
AIM/CGE 2.1       CD-LINKS_NPi2020_400 0           0.153213 -0.086132   
                                       1          -0.025027 -0.016988   
                                       2           0.164106 -0.065352   
                                       3           0.122711 -0.119902   
                                       4           0.143798 -0.097182   
...                                                     ...       ...   
WITCH-GLOBIOM 4.4 CD-LINKS_NPi2020_400 595         0.205725 -0.155350   
                                       596         0.245548 -0.010975   
                                       597         0.143254 -0.004426   
                                       598         0.167401 -0.042595   
                                       599         0.186103 -0.002185   

                                               change_non_co2  2015_warming  \
model             scenario             run_id                                 
AIM/CGE 2.1       CD-LINKS_NPi2020_400 0             0.024094      1.118762   
                                       1             0.054163      1.411990   
                                       2            -0.009230      1.256006   
                                       3            -0.081853      1.000904   
                                       4             0.035314      1.214386   
...                                                       ...           ...   
WITCH-GLOBIOM 4.4 CD-LINKS_NPi2020_400 595          -0.189144      1.110524   
                                       596          -0.053872      1.314478   
                                       597          -0.003011      1.245883   
                                       598          -0.142359      1.064670   
                                       599          -0.093717      1.093431   

                                               2100_warming  peak_warming  \
model             scenario             run_id                               
AIM/CGE 2.1       CD-LINKS_NPi2020_400 0           1.402425      1.625633   
                                       1           2.513044      2.516868   
                                       2           1.559434      1.804914   
                                       3           1.048721      1.375034   
                                       4           1.503015      1.715355   
...                                                     ...           ...   
WITCH-GLOBIOM 4.4 CD-LINKS_NPi2020_400 595         0.840507      1.390562   
                                       596         1.678354      1.937205   
                                       597         1.508828      1.663658   
                                       598         1.012923      1.369999   
                                       599         1.120199      1.394021   

                                               year_peak_warming  \
model             scenario             run_id                      
AIM/CGE 2.1       CD-LINKS_NPi2020_400 0                    2036   
                                       1                    2036   
                                       2                    2028   
                                       3                    2026   
                                       4                    2036   
...                                                          ...   
WITCH-GLOBIOM 4.4 CD-LINKS_NPi2020_400 595                  2060   
                                       596                  2069   
                                       597                  2069   
                                       598                  2069   
                                       599                  2082   

                                               cum_emissions_to_peak_GtCO2  \
model             scenario             run_id                                
AIM/CGE 2.1       CD-LINKS

<IPython.core.display.Javascript object>

Step 6: We probably want to add in the year of net zero CO2.

In [10]:
meta = pd.read_excel(
    Path(
        '../data/100_meta.xlsx'
    ),
    index_col=[0,1]
)

<IPython.core.display.Javascript object>

In [11]:
metrics.loc[:, 'netzero|CO2'] = (
    metrics.apply(
        lambda x: meta.loc[x.name[0:2], 'netzero|CO2'],
        axis=1
    )
)

<IPython.core.display.Javascript object>

Step 7: Write this out for further assessment.

In [12]:
metrics

drawdown_co2       ZEC  \
model             scenario             run_id                           
AIM/CGE 2.1       CD-LINKS_NPi2020_400 0           0.153213 -0.086132   
                                       1          -0.025027 -0.016988   
                                       2           0.164106 -0.065352   
                                       3           0.122711 -0.119902   
                                       4           0.143798 -0.097182   
...                                                     ...       ...   
WITCH-GLOBIOM 4.4 CD-LINKS_NPi2020_400 595         0.205725 -0.155350   
                                       596         0.245548 -0.010975   
                                       597         0.143254 -0.004426   
                                       598         0.167401 -0.042595   
                                       599         0.186103 -0.002185   

                                               change_non_co2  2015_warming  \
model             scenario             run_id                                 
AIM/CGE 2.1       CD-LINKS_NPi2020_400 0             0.024094      1.118762   
                                       1             0.054163      1.411990   
                                       2            -0.009230      1.256006   
                                       3            -0.081853      1.000904   
                                       4             0.035314      1.214386   
...                                                       ...           ...   
WITCH-GLOBIOM 4.4 CD-LINKS_NPi2020_400 595          -0.189144      1.110524   
                                       596          -0.053872      1.314478   
                                       597          -0.003011      1.245883   
                                       598          -0.142359      1.064670   
                                       599          -0.093717      1.093431   

                                               2100_warming  peak_warming  \
model             scenario             run_id                               
AIM/CGE 2.1       CD-LINKS_NPi2020_400 0           1.402425      1.625633   
                                       1           2.513044      2.516868   
                                       2           1.559434      1.804914   
                                       3           1.048721      1.375034   
                                       4           1.503015      1.715355   
...                                                     ...           ...   
WITCH-GLOBIOM 4.4 CD-LINKS_NPi2020_400 595         0.840507      1.390562   
                                       596         1.678354      1.937205   
                                       597         1.508828      1.663658   
                                       598         1.012923      1.369999   
                                       599         1.120199      1.394021   

                                               year_peak_warming  \
model             scenario             run_id                      
AIM/CGE 2.1       CD-LINKS_NPi2020_400 0                    2036   
                                       1                    2036   
                                       2                    2028   
                                       3                    2026   
                                       4                    2036   
...                                                          ...   
WITCH-GLOBIOM 4.4 CD-LINKS_NPi2020_400 595                  2060   
                                       596                  2069   
                                       597                  2069   
                                       598                  2069   
                                       599                  2082   

                                               cum_emissions_to_peak_GtCO2  \
model             scenario             run_id                                
AIM/CGE 2.1       CD-LINKS

<IPython.core.display.Javascript object>

In [13]:
metrics.to_csv(
    Path(
        '../data/401_lookup.csv'
    )
)

<IPython.core.display.Javascript object>